In [1]:
import warnings
warnings.filterwarnings("ignore")

from utils import dataset, load_bow
import pandas as pd
from datetime import datetime
from itertools import chain
import numpy as np

In [2]:
w = 1

In [3]:
col = [
    'Publication Type', \
    'Venue', \
    'Author Full Names', \
    'Article Title', \
    'Abstract', \
    'Publication Date', \
    'Publication Year']

venues = dataset()[col]
venues = venues.dropna(subset=['Abstract'])
venues['Abstract Lemmatized'] = load_bow('25_venues')
venues = venues.dropna(subset=['Publication Date'])
venues['Date'] = (venues['Publication Date']+'-'+venues['Publication Year']).apply(lambda x: datetime.strptime(x, '%b-%Y'))
query_dataframe = pd.read_csv('./database/query_dataframe.csv', index_col=1)
venues['Venue Full Name'] = venues['Venue'].apply(lambda x : query_dataframe.loc[x, 'Nome'])
venues = venues.sort_values(by='Date')
venues.index = range(len(venues))

In [4]:
authors = venues['Author Full Names']
authors = [names.split(';') for names in authors]
authors = [[name.strip().upper() for name in names] for names in authors]
authors = list(authors)
authors = list(chain.from_iterable(authors))
authors = sorted(list(set(authors)))

In [5]:
authors_dict = {}

for author in authors:
    authors_dict[author] = []

for index, row in venues.iterrows():
    for author in row['Author Full Names'].split(';'):
        authors_dict[author.upper().strip()].append(row['Article Title'])

In [6]:
authors_df = {k: str(v) for k, v in authors_dict.items()}
authors_df = pd.DataFrame(authors_df, index = ['Papers']).T
authors_df['publications'] = authors_df['Papers'].apply(lambda x: len(eval(x)))
authors_df.to_csv('./database/authors.csv')

In [44]:
lenght = 100
p = 10
while len(top) > lenght:
    top = authors_df.iloc[np.where(authors_df['publications'] >= p)].sort_index()
    top['pids'] = np.nan
    p += 1
    print(len(top))
top = authors_df.iloc[np.where(authors_df['publications'] >= p-2)].sort_index()
top['pids'] = np.nan
top.to_csv(f'./database/top_authors/top{lenght}authors.csv')

8396
7284
6360
5607
4976
4486
4059
3709
3404
3141
2920
2697
2490
2340
2172
2035
1923
1783
1701
1606
1534
1451
1374
1302
1233
1163
1100
1039
989
945
898
867
811
780
746
711
691
666
635
615
595
563
539
514
498
485
463
445
431
417
405
392
381
364
349
333
317
311
296
280
266
262
257
248
237
233
227
215
207
199
195
191
187
185
175
169
168
165
164
156
149
146
141
137
135
132
126
124
119
116
114
110
107
102
98


In [37]:
top = authors_df.iloc[np.where(authors_df['publications'] >= 25)].sort_index()
len(top)

2035

In [34]:
p

27